In [1]:
import pandas as pd
import numpy as np

In [2]:
#!pip install haversine

In [3]:
from azureml.core import Datastore,Workspace,Dataset
# Get a named datastore from the current workspace
ws = Workspace.from_config()
datastore = Datastore.get(ws, datastore_name='aisdatastore')

In [4]:
rotte_dataset = ws.datasets["rotte"]

In [5]:
from datetime import datetime

def preprocessRotte(df):

    df = df.to_pandas_dataframe()
    #drop NA values
    df=df.dropna()
    #build timestamp column to be added to rotte_092021 dataframe
    stamp=df['stamp'].astype(float)
    #stamp    
    timestamp_column = stamp.apply(lambda x: datetime.fromtimestamp(x))
    #timestamp_column
    #add timestamp column to rotte_092021 dataframe
    df['timestamp'] = timestamp_column.values
    #sort_dataset by mmsi,stamp,lng,lat
    df = df.sort_values(by=['mmsi','stamp','lng','lat'],ascending=[True,True,True,True])
    
    return df

In [6]:
rotte_dataset = preprocessRotte(rotte_dataset)

Resolving access token for scope "https://datalake.azure.net//.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration


In [8]:
ais_dataset =  ws.datasets["ais"]

In [9]:
shiptypeDict = {
    "0": "Unspecified",
    "1": "NULL",
    "2": "Fishing",
    "3": "Search and Rescue/Special Craft/Tug/Passenger/Other",
    "4": "High Speed Craft",
    "6": "Passenger",
    "7": "Cargo",
    "8": "Tanker",
    "9": "Sailing Vessel/Pleasure Craft"
}


In [10]:
def setShipTypeDesc (row):
    if row['type_summary'] == 'NULL':
        row['shiptype_desc'] = shiptypeDict[str(row['shiptype'])]
    else:
        row['shiptype_desc'] = row['type_summary']

In [11]:
def preprocessAIS(df):

    df = df.to_pandas_dataframe()
    #set type when is NULL
    df['shiptype_desc'] = df.apply(lambda x: (shiptypeDict[str(x.shiptype)] if x.type_summary == 'NULL' else x.type_summary), axis=1)
    #keep vessels whose type != Fishing
    df = df[df['shiptype']!=2]
    
    return df

In [12]:
ais_dataset = preprocessAIS(ais_dataset)

In [13]:
vessels_type = ais_dataset['type_summary'].unique()
vessels_flag = ais_dataset['flag'].unique()
print(vessels_type)
print(vessels_flag)

['Cargo' 'Tug' 'Pleasure Craft' 'NULL' 'Passenger' 'Tanker'
 'Special Craft' 'High Speed Craft' 'Sailing Vessel' 'Search and Rescue'
 'Other' 'Unspecified']
['--' 'KY' 'IT' 'GB' 'DZ' 'MH' 'AL' 'BE' 'BG' 'CY' 'DE' 'MD' 'MT' 'DK'
 'ES' 'FR' 'GI' 'GR' 'HR' 'MA' 'NL' 'PA' 'KI' 'RU' 'BS' 'LU' 'MC' 'PT'
 'NO' 'SE' 'CH' 'TR' 'UA' 'LV' 'LT' 'AG' 'BM' 'BZ' 'BB' 'HN' 'US' 'JM'
 'KN' 'VC' 'VG' 'SA' 'CN' 'TW' 'AZ' 'IL' 'JP' 'KZ' 'KR' 'KW' 'LB' 'QA'
 'HK' 'PW' 'CK' 'NU' 'PH' 'SG' 'TV' 'VN' 'VU' 'WF' 'CM' 'KM' 'EG' 'LR'
 'PL' 'LY' 'SL' 'TG' 'TN']


In [14]:
vessels_mmsi = ais_dataset['mmsi'].unique().astype(int)
vessels_imo = ais_dataset['imo'].unique().astype(int)

print(len(vessels_mmsi))
print(len(vessels_imo))

3150
3122


In [15]:
world_porti = ws.datasets["porti1"]

In [16]:
def preprocessPorts(df):

    try:
        df = df.to_pandas_dataframe()
    except Exception as e:
        print(e)
    df = df[['Country','Name','Latitude','Longitude','UNLocode']]
    df.reset_index(drop=True,inplace=True)
    return df


In [17]:
world_porti = preprocessPorts(world_porti)

In [18]:
#Drop rows whose mmsi is not in rotte_092021_simplified(rotte_092021_mmsi,vessels_mmsi) from rotte_092021,
# i.e. get only rows whos mmsi is in mmsi_intersection

rotte_dataset_simplified = rotte_dataset[rotte_dataset['mmsi'].astype(int).isin(vessels_mmsi)]
#rotte_dataset_simplified.head()
print(len(rotte_dataset_simplified))
rotte_dataset_simplified = rotte_dataset_simplified.reset_index()
#rotte_dataset_simplified.head()

33334193


ELABORAZIONE DELLE ROTTE

In [19]:
from datetime import datetime
import haversine as hs
from haversine import Unit

In [20]:
#elaborazione degli arrivi (df)
def arrival_elaboration(df_rotte):

    dim=len(df_rotte)
    print("df_rotte len before: ",dim)

    df_arrival = pd.DataFrame(columns = ['row','mmsi','arrival','departure','lng','lat','lng_orig','lat_orig','speed','status'])

    oldmmsi=0
    sumrec=0
    start=0
    lat_orig=0
    lng_orig=0
    oldlng=0
    oldlat=0
    start = None
    i=0
    status=0 ###0-nuovo 1-arrivato 2-partito
    end = None
    speed = None

    present = 0 # already present in the port, only for first occurrence of the ship 0: false, 1: true

    start_time = datetime.now()

    for item in df_rotte.itertuples():
        try:
            #item = df_rotte.iloc[i]
            mmsi,time_voyage,lng,lat,speed = item.mmsi,item.timestamp,float(item.lng),float(item.lat),int(item.speed)

            if(mmsi!=oldmmsi):
                if (status==1):#si riferisce alla old ship
                    df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                           'departure':end,'lng':oldlng,'lat':oldlat,'lng_orig':lng_orig,'lat_orig':lat_orig,
                            'speed':speed,'status':status},ignore_index=True)
                start,status,lng_orig,lat_orig,oldlng,oldlat,oldmmsi=None,0,0,0,0,0,mmsi
                oldmmsi = mmsi
    
            if(speed==0):
                if (status==0):
                    start,oldlng,oldlat,present = time_voyage,lng,lat,1
                
                #To calculate distance in meters
                if(status<2):
                    loc1=(lat,lng)
                    loc2=(oldlat,oldlng)
            
                    distance = hs.haversine(loc1,loc2,unit=Unit.METERS)
            
                    if(distance > 3000.0):
                        #print(distance)
                        #if ((abs(oldlng-lng)+abs(oldlat-lat))>0.3):
                        if(present == 1):
                            status = 0
                        df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                           'departure':end,'lng':oldlng,'lat':oldlat,
                                       'lng_orig':lng_orig,'lat_orig':lat_orig,'speed':speed,'status':status},ignore_index=True)
                        start,lng_orig,lat_orig,oldlng,oldlat,present = time_voyage,oldlng,oldlat,lng,lat,0
                    
                if(status==2):
                    start,lng_orig,lat_orig,oldlng,oldlat = time_voyage,oldlng,oldlat,lng,lat
                    
                end=time_voyage
                #after first occurrence of the ship,status = arrived
                status=1
                
            if (speed>0):
                if(status==1):

                    loc1=(lat,lng)
                    loc2=(oldlat,oldlng)
                    distance = hs.haversine(loc1,loc2,unit=Unit.METERS)
                    if(distance > 3000.0):
                        status = 2
                        if(present == 1):
                            status = 0
                        df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                           'departure':end,'lng':oldlng,'lat':oldlat,
                            'lng_orig':lng_orig,'lat_orig':lat_orig,'speed':speed,'status':status},ignore_index=True)
                        
                        status,lng_orig,lat_orig,present = 2,oldlng,oldlat,0
                if(status==0):
                    status=2
                   
            #stampa di controllo
            if(i%1000000 == 0):
               print(i)
        except Exception as e:
            print(i," ")
            print(e)
        i+=1
    #end for
    
    #scrive l'ultimo record
    if(status==1):
            df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                        'departure':end,'lng':oldlng,'lat':oldlat,
                        'lng_orig':lng_orig,'lat_orig':lat_orig,'speed':speed,'status':status},ignore_index=True)

 
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("df_arrival len after: ",len(df_arrival))
    
    return(df_arrival)

In [21]:
df_rotte = rotte_dataset_simplified[['mmsi','stamp','timestamp','lng','lat','speed']]
df_rotte = df_rotte.sort_values(by=['mmsi','stamp'],ascending=[True,True])
df_rotte = df_rotte.reset_index()

In [22]:
df_arrival= arrival_elaboration(df_rotte)

df_rotte len before:  33334193
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
Duration: 0:17:13.004107
df_arrival len after:  82808


In [30]:
ita_ports = world_porti[world_porti['Country']=='ITA']
#print(len(ita_ports))
foreign_ports = world_porti[world_porti['Country'] !='ITA']
#print(len(foreign_ports))
world_porti = ita_ports.append(foreign_ports,ignore_index=True)
print(len(world_porti))
world_porti.reset_index()


13763


,index,Country,Name,Latitude,Longitude,UNLocode
0,0,ITA,Fusina,45.422091,12.256319,ITFSA
1,1,ITA,Genova,44.399822,8.924361,ITGOA
2,2,ITA,Marghera,45.465793,12.233153,ITPMA
3,3,ITA,Rinella,38.546404,14.828765,ITRIN
4,4,ITA,Santa Panagia,37.114285,15.251981,ITSPA
...,...,...,...,...,...,...
13758,13758,TON,Nuku'alofa,-21.136978,-175.181200,None
13759,13759,TON,Pangai,-19.803124,-174.352489,None
13760,13760,TKL,Atafu,-8.542822,-172.517323,None
13761,13761,TKL,Fakaofo,-9.385887,-171.248517,None


In [31]:
def port_assign(df_rotte,df_porti):
    
    n=len(df_rotte)
    df = df_rotte
    df["port"] = ["null"]*n
    df["port_orig"] = ["null"]*n
    df["UNLocode"] = ["NA"]*n 
    df["UNLocode_orig"] = ["NA"]*n

    start_time = datetime.now()
    
    i=0
    for rotta in df_rotte.itertuples():
        found_orig,found=0,0
        #departure port position
        loc_orig=(float(rotta.lat_orig),float(rotta.lng_orig))
        #arrival port position
        loc=(float(rotta.lat),float(rotta.lng))
        
        for porto in df_porti.itertuples():            
            loc_port=(float(porto.Latitude),float(porto.Longitude))            
            distance1 = hs.haversine(loc,loc_port,unit=Unit.METERS)
            if(distance1 < 5000.0):
                df.loc[i,'port']=porto.Name
                df.loc[i,'UNLocode'] = porto.UNLocode
                found=1
                break
        if (found==0):
            df.loc[i,'port']='Not found'
        
        for porto in df_porti.itertuples():             
            loc_port=(float(porto.Latitude),float(porto.Longitude))
            distance2 = hs.haversine(loc_orig,loc_port,unit=Unit.METERS)
            if(distance2 < 5000.0):
                df.loc[i,'port_orig']=porto.Name
                df.loc[i,'UNLocode_orig'] = porto.UNLocode
                found_orig=1
                break    

        if (found_orig==0):
            df.loc[i,'port_orig']='Not found'
            
        #stampa di controllo
        if(i%10000 == 0):
               print(i)
        i+=1
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("df_arrival len after: ",len(df))
        
    return(df)

In [32]:
df_rotte_arrivi_porti = port_assign(df_arrival,world_porti)

0
10000
20000
30000
40000
50000
60000
70000
80000
Duration: 0:36:28.003272
df_arrival len after:  82808


In [34]:
#export all records 1st and 2nd step
df_rotte_arrivi_porti.to_csv("df_rotte_arrivi_porti_set2021_after_portAssign.csv",index=False,sep=',')
#df_rotte_arrivi_porti.head()

In [59]:
df_rotte_arrivi_porti_pre = pd.read_csv("df_rotte_arrivi_porti_set2021_after_portAssign.csv", sep=',')

ARRIVI PARTENZE

In [60]:
df_rotte_arrivi_porti[["arrival", "departure"]] = df_rotte_arrivi_porti[["arrival", "departure"]].apply(pd.to_datetime)

In [61]:
def calc_tratte(df_arrivi):

    dim=len(df_arrivi)
    print("dim arrivals: ",dim)

    df_arrivi[["arrival", "departure"]] = df_arrivi[["arrival", "departure"]].apply(pd.to_datetime)


    df_tratte = pd.DataFrame(columns = ['mmsi','partenza','porto_orig','cod_orig','arrivo','porto_dest','cod_dest','status'])

    prevmmsi=0

    i=0
    status=0 ###
    prevpartenza=""
    prevport=""
    prevcode=""

    start_time = datetime.now()

    for current in df_arrivi.itertuples():
        i=i+1

        if (current.mmsi!=prevmmsi):
            if ((prevpartenza!="") & (status == 2)):                
                df_tratte=df_tratte.append({'mmsi':prevmmsi,'partenza':prevpartenza,
                    'porto_orig':prevport,'cod_orig':prevcode,'arrivo':"",
                    'porto_dest':"",'cod_dest':"",'status':status},ignore_index=True)
            prevpartenza=""
            prevport=""
            prevcode=""
            status = current.status

        time_approdo=(current.departure-current.arrival).seconds
        if (time_approdo>=900 and current.port!="Not found"):
            #print("rec ",i," : ",current)
            if(current.status != 0):

                df_tratte=df_tratte.append({'mmsi':current.mmsi,'partenza':prevpartenza,
                    'porto_orig':prevport,'cod_orig':prevcode,'arrivo':current.arrival,
                    'porto_dest':current.port,'cod_dest':current.UNLocode,'status':current.status},ignore_index=True)

            prevmmsi=current.mmsi
            prevpartenza=current.departure
            prevport=current.port
            prevcode=current.UNLocode
            status = current.status

        
        if(i%10000 == 0):
               print(i)

    if ((prevpartenza!="") & (status == 2)):
        
        df_tratte=df_tratte.append({'mmsi':prevmmsi,'partenza':prevpartenza,
            'porto_orig':prevport,'cod_orig':prevcode,'arrivo':"",
            'porto_dest':"",'cod_dest':"",'status':status},ignore_index=True)


    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("df_arrival len after: ",len(df_tratte))
    
    return(df_tratte)

In [62]:
df_rotte_arrivi_porti=calc_tratte(df_rotte_arrivi_porti)

dim arrivals:  82808
10000
20000
30000
40000
50000
60000
70000
80000
Duration: 0:04:27.719430
df_arrival len after:  39451


ADD INFORMATION TO THE DATASET

In [63]:
vessels_mmsi_imo_dict = pd.Series(ais_dataset.imo.values,index=ais_dataset.mmsi).to_dict()
#vessels_mmsi_callsign_dict = pd.Series(ais_dataset.callsign.values,index=ais_dataset.mmsi).to_dict()
vessels_mmsi_shiptype_desc_dict = pd.Series(ais_dataset.shiptype_desc.values,index=ais_dataset.mmsi).to_dict()
vessels_mmsi_type_summary_dict = pd.Series(ais_dataset.type_summary.values,index=ais_dataset.mmsi).to_dict()

vessels_mmsi_imo_clean_dict = {k: vessels_mmsi_imo_dict[k] for k in vessels_mmsi_imo_dict if not pd.isna(vessels_mmsi_imo_dict[k])}
#vessels_mmsi_callsign_clean_dict = {k: vessels_mmsi_callsign_dict[k] for k in vessels_mmsi_callsign_dict if not pd.isna(vessels_mmsi_callsign_dict[k])}
vessels_mmsi_shiptype_desc_clean_dict = {k: vessels_mmsi_shiptype_desc_dict[k] for k in vessels_mmsi_shiptype_desc_dict if not pd.isna(vessels_mmsi_shiptype_desc_dict[k])}
vessels_mmsi_type_summary_clean_dict = {k: vessels_mmsi_type_summary_dict[k] for k in vessels_mmsi_type_summary_dict if not pd.isna(vessels_mmsi_type_summary_dict[k])}

In [64]:
df_rotte_arrivi_porti["imo"] = df_rotte_arrivi_porti["mmsi"].astype(int)
df_rotte_arrivi_porti['imo'].replace(vessels_mmsi_imo_clean_dict ,inplace=True)

#df_rotte_arrivi_porti["callsign"] = df_rotte_arrivi_porti["mmsi"].astype(int)
#df_rotte_arrivi_porti['callsign'].replace(vessels_mmsi_callsign_clean_dict ,inplace=True)

df_rotte_arrivi_porti['shiptype_desc'] = df_rotte_arrivi_porti["mmsi"].astype(int)
df_rotte_arrivi_porti['shiptype_desc'].replace(vessels_mmsi_shiptype_desc_clean_dict ,inplace=True)

df_rotte_arrivi_porti['type_summary'] = df_rotte_arrivi_porti["mmsi"].astype(int)
df_rotte_arrivi_porti['type_summary'].replace(vessels_mmsi_type_summary_clean_dict ,inplace=True)

In [66]:
#export all records
df_rotte_arrivi_porti.to_csv("df_rotte_arrivi_porti_set2021_final.csv",index=False,sep=',')
#df_rotte_arrivi_porti

In [68]:
df_rotte_arrivi_porti[df_rotte_arrivi_porti['mmsi']==209156000  ]

,mmsi,partenza,porto_orig,cod_orig,arrivo,porto_dest,cod_dest,status,imo,shiptype_desc,type_summary
332,209156000.0,2021-10-04 03:29:39,Cagliari,ITCAG,2021-10-04 04:39:22,Cagliari,ITCAG,2,9287340,Cargo,Cargo
333,209156000.0,2021-10-13 09:19:12,Cagliari,ITCAG,2021-10-23 02:23:41,Cartagena,ESCAR,2,9287340,Cargo,Cargo
334,209156000.0,2021-10-29 09:26:52,Cartagena,ESCAR,,,,2,9287340,Cargo,Cargo


COMPUTE STATISTICS OF ARRIVALS IN THE PORTS

In [69]:
#df_rotte_arrivi_porti = pd.read_csv("df_rotte_arrivi_porti_set2021_final.csv", sep=',')
df_rotte_arrivi_porti.head()

,mmsi,partenza,porto_orig,cod_orig,arrivo,porto_dest,cod_dest,status,imo,shiptype_desc,type_summary
0,1193046.0,,,,2021-09-01 01:29:26,Porto Torres,ITPTO,1,7512507,Tug,Tug
1,1193046.0,2021-09-01 01:50:28,Porto Torres,ITPTO,2021-09-01 02:26:27,Porto Torres,ITPTO,1,7512507,Tug,Tug
2,1193046.0,2021-09-01 02:41:28,Porto Torres,ITPTO,2021-09-01 04:23:29,Porto Torres,ITPTO,1,7512507,Tug,Tug
3,1193046.0,2021-09-01 04:44:31,Porto Torres,ITPTO,2021-09-01 05:35:31,Porto Torres,ITPTO,1,7512507,Tug,Tug
4,1193046.0,2021-09-01 20:20:53,Porto Torres,ITPTO,2021-09-01 20:38:53,Porto Torres,ITPTO,2,7512507,Tug,Tug


In [92]:
df_set = df_rotte_arrivi_porti[pd.DatetimeIndex(df_rotte_arrivi_porti['partenza']).month == 9]

df_ott = df_rotte_arrivi_porti[pd.DatetimeIndex(df_rotte_arrivi_porti['partenza']).month == 10]

In [93]:
df = df_set[df_set['cod_orig'].str.startswith('IT', na=False)].groupby(['cod_orig','porto_orig']).size().reset_index(name='Size')
df.to_csv('rotte_arrivi_porti_set2021_tot_port_orig.csv',index=False,sep=',')

df = df_ott[df_ott['cod_orig'].str.startswith('IT', na=False)].groupby(['cod_orig','porto_orig']).size().reset_index(name='Size')
df.to_csv('rotte_arrivi_porti_ott2021_tot_port_orig.csv',index=False,sep=',')

In [94]:
df = df_set[df_set['cod_dest'].str.startswith('IT', na=False)].groupby(['cod_dest','porto_dest']).size().reset_index(name='Size')
#df = df.groupby(['cod_dest','porto_dest']).size().reset_index(name='Size')
df.to_csv('rotte_arrivi_porti_set2021_tot_port_dest.csv',index=False,sep=',')

df = df_ott[df_ott['cod_dest'].str.startswith('IT', na=False)].groupby(['cod_dest','porto_dest']).size().reset_index(name='Size')
df.to_csv('rotte_arrivi_porti_ott2021_tot_port_dest.csv',index=False,sep=',')